Post new ads or change ads one piece at a time

In [1]:
import pandas as pd
import pickle

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
from selenium.webdriver.chrome.options import Options

In [3]:
YA = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
opts = Options()
opts.add_argument("user-agent=YA")
opts.add_argument('--disable-extensions')
opts.add_argument("start-maximized")

In [4]:
#reload dataframe with cols: location, url, zip_code, ad_type
df = pd.read_pickle('clist_df')

In [5]:
#obtain login info and ad content
#*Note: user should define these variables directly with their own login info and ad content
emails = pickle.load(open('email_info','rb'))
passwords = pickle.load(open('pword_info','rb'))
post_title = pickle.load(open('post_title','rb'))
ad = pickle.load(open('ad_info','rb'))
phone_number = pickle.load(open('phone_info','rb'))
contact_name = pickle.load(open('contact_name','rb'))

In [6]:
#define college towns and highschool towns
col_towns = df[df['ad_type']=='col']
col = list(col_towns.index)
hs_towns = df[df['ad_type']=='hs']
hs = list(hs_towns.index)

In [7]:
#function to login and post to a particular location
#*note: user will need to change the first two xpaths to correspond with the specific ad type they would like to post
def post(loc, email, password):
    link = df['url'][loc]
    zipcode = df['zip_code'][loc]
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element_by_id('post').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
    driver.find_element_by_id('inputEmailHandle').send_keys(email)
    driver.find_element_by_id('inputPassword').send_keys(password)
    driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()
    try:
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    except:
        #for those locations with an extra landing page, just choose the first sublocation
        #*idea: consider discovering ones w extra page and looping thru sublocations
        driver.find_element_by_xpath('/html/body/article/section/form/ul/li[1]/label').click()
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    driver.find_element_by_id('postal_code').send_keys(zipcode)
    driver.find_element_by_id('PostingBody').send_keys(ad)
    #*option: for longer ads, consider using copy and paste by uncommenting the line below
    # driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')
    driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
    driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
    driver.find_element_by_xpath('//*[@id="postingForm"]/div/button').click()
    driver.find_element_by_xpath('//*[@id="leafletForm"]/button[1]').click()
    driver.find_element_by_xpath('/html/body/article/section/form/button').click()
    #*option: to automatically post without first reviewing the content, uncomment the line below
#     driver.find_element_by_xpath('//*[@id="publish_top"]/button').click()

In [18]:
loc = hs[-12]
email = emails[2]
password = passwords[2]
post_title = 'SAT & ACT Expert | 10 yrs Experience | Affordable Rates!' 

In [19]:
link = df['url'][loc]
zipcode = df['zip_code'][loc]
driver = webdriver.Chrome(chrome_options=opts)
driver.wait = WebDriverWait(driver, 15)
driver.get(link)

driver.find_element_by_id('post').click()
driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()
driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
driver.find_element_by_id('inputEmailHandle').send_keys(email)
driver.find_element_by_id('inputPassword').send_keys(password)
driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()

driver.find_element_by_id('PostingTitle').send_keys(post_title)
driver.find_element_by_id('postal_code').send_keys(zipcode)
# driver.find_element_by_id('PostingBody').send_keys(ad)                # load ad from pickle
driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')      # or paste the ad from clipboard

driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
driver.find_element_by_xpath('//*[@id="postingForm"]/div/button').click()